# 12-3 1D CNN으로 IMDB 리뷰 분류하기

In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)

/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [7]:
max_len = 200
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

print('X_train의 크기(shape) :',X_train.shape)
print('X_test의 크기(shape) :',X_test.shape)

X_train의 크기(shape) : (25000, 200)
X_test의 크기(shape) : (25000, 200)


In [8]:
y_train[:5]

array([1, 0, 0, 1, 0])

## 1D CNN으로 IMDB 리뷰 분류하기

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 256))
model.add(Dropout(0.3))
model.add(Conv1D(256, 3, padding='valid', activation='relu')) # 커널수=256, 커널크기=3
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [11]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(X_train, y_train, epochs = 20, validation_data = (X_test, y_test), callbacks=[es, mc])

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.4174 - acc: 0.7973
Epoch 00001: val_acc improved from -inf to 0.88184, saving model to best_model.h5
782/782 [==============================] - 15s 19ms/step - loss: 0.4174 - acc: 0.7973 - val_loss: 0.2816 - val_acc: 0.8818
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.2214 - acc: 0.9142
Epoch 00002: val_acc improved from 0.88184 to 0.88680, saving model to best_model.h5
782/782 [==============================] - 15s 19ms/step - loss: 0.2214 - acc: 0.9142 - val_loss: 0.2693 - val_acc: 0.8868
Epoch 3/20
781/782 [============================>.] - ETA: 0s - loss: 0.1154 - acc: 0.9594
Epoch 00003: val_acc did not improve from 0.88680
782/782 [==============================] - 15s 19ms/step - loss: 0.1154 - acc: 0.9594 - val_loss: 0.2983 - val_acc: 0.8854
Epoch 4/20
781/782 [============================>.] - ETA: 0s - loss: 0.0522 - acc: 0.9834
Epoch 00004: val_acc did not improve from 0.88680


In [12]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 2s 3ms/step - loss: 0.2693 - acc: 0.8868

 테스트 정확도: 0.8868
